In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score, classification_report
from helpers import Load_model, Predict, Eval_against_gold, Is_english,Row_to_vector

In [2]:
model, tokenizer, frames, threshold = Load_model("best")

Model, tokenizer, frames, and threshold loaded from: best
Using device: cuda


In [3]:
gold_df_mickey = pd.read_excel('data/gold_standard_labeled_mickey.xlsx')
gold_df_mickey = gold_df_mickey[gold_df_mickey["content"].str.split().str.len() >=100]
gold_df_mickey = gold_df_mickey[gold_df_mickey["content"].progress_apply(Is_english)]

100%|█████████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 152.48it/s]


In [4]:
gold_df_matt = pd.read_excel('data/gold_standard_labeled_matt.xlsx')
gold_df_matt = gold_df_matt[gold_df_matt["content"].str.split().str.len() >=100]
gold_df_matt = gold_df_matt[gold_df_matt["content"].progress_apply(Is_english)]

100%|█████████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 260.14it/s]


In [5]:
gold_df_mark = pd.read_csv('data/gold_standard_labeled_mark.csv',encoding='cp1252')
gold_df_mark = gold_df_mark[gold_df_mark["content"].str.split().str.len() >=100]
gold_df_mark = gold_df_mark[gold_df_mark["content"].progress_apply(Is_english)]

100%|█████████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 259.26it/s]


In [6]:
# Compare gold standard labels
agreement_rows = []

for i in range(len(gold_df_mickey)):
    
    uuid = gold_df_mickey.iloc[i]["uuid"]
    v_mickey = Row_to_vector(gold_df_mickey.iloc[i])
    v_matt   = Row_to_vector(gold_df_matt.iloc[i])
    v_mark   = Row_to_vector(gold_df_mark.iloc[i])
    content = gold_df_mickey.iloc[i]['content']

    # Build vectors for one, two, or all three of us assinging the same label
    counts = v_mickey + v_matt + v_mark        
    union_vec        = (counts >= 1).astype(int)
    two_plus_vec     = (counts >= 2).astype(int)
    intersection_vec = (counts == 3).astype(int)

    agreement_rows.append({
        "uuid": uuid,
        "union": union_vec,
        "2+": two_plus_vec,
        "intersection": intersection_vec,
        'content': content
    })


agreement_df = pd.DataFrame(agreement_rows)
agreement_df.head()

,uuid,union,2+,intersection,content
0,15179634683817202110,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0]","[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","If there’s a war on women, the women may be wi..."
1,6483872619100768126,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Twitter user @CurvedDaily pulled off an amazin...
2,14295426020314347428,"[0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","interview visit, mount holyoke college gates\n..."
3,8766252870932411430,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Oil prices jumped more than 5 percent on Frida...
4,14630145903893105805,"[0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","For years, Sohail Zafar Chattha, a seasoned Pa..."


In [7]:
#Eval individual labels vs models predictions 
y_true, y_pred = Eval_against_gold(gold_df_mickey)
print("Classification report for Mickey:")
print(classification_report(y_true, y_pred, target_names=frames))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 76.92it/s]

Evaluated articles: 74
Classification report for Mickey:
              precision    recall  f1-score   support

     cap&res       0.33      0.17      0.22        18
       crime       0.69      0.90      0.78        20
     culture       0.58      0.35      0.44        20
    economic       1.00      0.59      0.74        27
    fairness       0.52      0.86      0.65        14
      health       0.77      0.77      0.77        13
    legality       0.33      1.00      0.49        16
    morality       1.00      0.06      0.11        18
      policy       0.23      0.77      0.36        13
   political       0.81      0.48      0.60        27
   public_op       0.55      0.71      0.62        17
quality_life       0.42      0.53      0.47        19
  regulation       0.29      0.50      0.36         8
    security       0.72      0.82      0.77        22

   micro avg       0.51      0.60      0.55       252
   macro avg       0.59      0.61      0.53       252
weighted avg       0.63

In [8]:
#Eval individual labels vs models predictions 
y_true, y_pred = Eval_against_gold(gold_df_matt)
print("Classification report for Matt:")
print(classification_report(y_true, y_pred, target_names=frames))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 76.92it/s]

Evaluated articles: 74
Classification report for Matt:
              precision    recall  f1-score   support

     cap&res       0.22      0.25      0.24         8
       crime       0.38      0.91      0.54        11
     culture       0.58      0.30      0.40        23
    economic       1.00      0.73      0.84        22
    fairness       0.30      0.70      0.42        10
      health       0.54      0.70      0.61        10
    legality       0.33      0.94      0.48        17
    morality       1.00      0.07      0.12        15
      policy       0.30      0.68      0.42        19
   political       0.75      0.41      0.53        29
   public_op       0.23      0.56      0.32         9
quality_life       0.21      0.42      0.28        12
  regulation       0.29      0.33      0.31        12
    security       0.56      0.88      0.68        16

   micro avg       0.41      0.56      0.47       213
   macro avg       0.48      0.56      0.44       213
weighted avg       0.54  

In [9]:
#Eval individual labels vs models predictions 
y_true, y_pred = Eval_against_gold(gold_df_mark)
print("Classification report for Mark:")
print(classification_report(y_true, y_pred, target_names=frames))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 83.34it/s]

Evaluated articles: 74
Classification report for Mark:
              precision    recall  f1-score   support

     cap&res       0.11      0.20      0.14         5
       crime       0.42      1.00      0.59        11
     culture       0.67      0.42      0.52        19
    economic       0.75      0.75      0.75        16
    fairness       0.43      0.62      0.51        16
      health       0.46      1.00      0.63         6
    legality       0.06      1.00      0.12         3
    morality       1.00      0.07      0.13        14
      policy       0.09      0.80      0.17         5
   political       0.50      0.73      0.59        11
   public_op       0.27      0.67      0.39         9
quality_life       0.08      0.40      0.14         5
  regulation       0.14      0.67      0.24         3
    security       0.36      1.00      0.53         9

   micro avg       0.28      0.63      0.39       132
   macro avg       0.38      0.67      0.39       132
weighted avg       0.50  


C:\Users\Mickey\anaconda3\envs\milestone3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [10]:
#Eval pre-vectorized labels vs models predictions 
vector_columns = 'union', '2+','intersection'
for col in vector_columns:
    y_true, y_pred = Eval_against_gold(agreement_df,vector_col=col)
    
    print(f"Classification report:{col}")
    print(classification_report(y_true, y_pred, target_names=frames))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 90.92it/s]


Evaluated articles: 74
Classification report:union
              precision    recall  f1-score   support

     cap&res       0.44      0.21      0.29        19
       crime       0.73      0.90      0.81        21
     culture       0.83      0.32      0.47        31
    economic       1.00      0.55      0.71        29
    fairness       0.65      0.60      0.62        25
      health       0.77      0.71      0.74        14
    legality       0.41      0.95      0.57        21
    morality       1.00      0.04      0.08        24
      policy       0.40      0.74      0.52        23
   political       0.81      0.41      0.54        32
   public_op       0.64      0.58      0.61        24
quality_life       0.46      0.46      0.46        24
  regulation       0.36      0.36      0.36        14
    security       0.76      0.83      0.79        23

   micro avg       0.59      0.54      0.56       324
   macro avg       0.66      0.55      0.54       324
weighted avg       0.69      

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 83.33it/s]


Evaluated articles: 74
Classification report:2+
              precision    recall  f1-score   support

     cap&res       0.11      0.10      0.11        10
       crime       0.54      0.93      0.68        15
     culture       0.58      0.35      0.44        20
    economic       1.00      0.70      0.82        23
    fairness       0.35      0.89      0.50         9
      health       0.62      0.80      0.70        10
    legality       0.27      1.00      0.42        13
    morality       1.00      0.06      0.11        17
      policy       0.21      0.75      0.33        12
   political       0.75      0.50      0.60        24
   public_op       0.32      0.78      0.45         9
quality_life       0.17      0.40      0.24        10
  regulation       0.29      0.50      0.36         8
    security       0.60      0.88      0.71        17

   micro avg       0.41      0.60      0.49       197
   macro avg       0.49      0.62      0.46       197
weighted avg       0.56      0.6

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 90.92it/s]

Evaluated articles: 74
Classification report:intersection
              precision    recall  f1-score   support

     cap&res       0.11      0.50      0.18         2
       crime       0.23      1.00      0.38         6
     culture       0.42      0.45      0.43        11
    economic       0.75      0.92      0.83        13
    fairness       0.26      1.00      0.41         6
      health       0.38      1.00      0.56         5
    legality       0.04      1.00      0.08         2
    morality       1.00      0.17      0.29         6
      policy       0.02      0.50      0.04         2
   political       0.50      0.73      0.59        11
   public_op       0.09      1.00      0.17         2
quality_life       0.08      1.00      0.15         2
  regulation       0.07      1.00      0.13         1
    security       0.28      1.00      0.44         7

   micro avg       0.20      0.78      0.32        76
   macro avg       0.30      0.81      0.33        76
weighted avg       0.4


C:\Users\Mickey\anaconda3\envs\milestone3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
